In [1]:
!pip install spacy pandas scikit-learn streamlit pyngrok openpyxl
!python -m spacy download en_core_web_sm


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 77.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 109.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 129.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [2]:
import spacy
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re, os


In [3]:
!pip install pdfplumber

import pdfplumber

def extract_text_from_pdf(file_path):
    text = ""
    with pdfplumber.open(file_path) as pdf:
        for page in pdf.pages:
            text += page.extract_text() or ""
    return text


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 87.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 108.4 MB/s eta 0:00:00


In [4]:
nlp = spacy.load("en_core_web_sm")

def preprocess(text):
    doc = nlp(text.lower())
    tokens = [token.lemma_ for token in doc
              if not token.is_stop and not token.is_punct]
    return " ".join(tokens)


In [5]:
job_description = """We are looking for a Python Developer with experience in NLP,
machine learning, and building web applications using Flask/Streamlit."""

job_desc_clean = preprocess(job_description)


In [6]:
def rank_resumes(resume_texts, job_desc):
    corpus = [job_desc] + resume_texts
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(corpus)

    job_vec = tfidf_matrix[0]
    resume_vecs = tfidf_matrix[1:]

    scores = cosine_similarity(job_vec, resume_vecs)[0]
    return scores


In [7]:
from google.colab import files
uploaded = files.upload()

resume_texts = []
resume_names = []

for filename in uploaded.keys():
    text = extract_text_from_pdf(filename)
    cleaned = preprocess(text)
    resume_texts.append(cleaned)
    resume_names.append(filename)


Saving Borigi-Jyothiradhithya-FlowCV-Resume-20250831 (2).pdf to Borigi-Jyothiradhithya-FlowCV-Resume-20250831 (2).pdf


In [8]:
scores = rank_resumes(resume_texts, job_desc_clean)
results = pd.DataFrame({
    "Resume": resume_names,
    "Score": scores
}).sort_values(by="Score", ascending=False)

results.reset_index(drop=True, inplace=True)
results


,Resume,Score
0,Borigi-Jyothiradhithya-FlowCV-Resume-20250831 ...,0.168262


In [9]:
results.to_csv("ranked_candidates.csv", index=False)
files.download("ranked_candidates.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>